In [ ]:
from src import api
from src import wiki

In [ ]:
url = "https://lolpros.gg/player/goksi"
player_data = api.get_player_data_by_lolpros(url, last_n=20)

In [ ]:
team_data = wiki.get_team_data_from_lol_wiki("sampi")